### Package version
- tensorflow==2.2.0rc2
- tensorboard==2.2.0

### Load tensorflow

In [1]:
import tensorflow as tf
tf.__version__

'2.2.0-rc2'

### Collect Data

In [2]:
(trainX, trainY),(testX, testY) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
trainX.shape

(60000, 28, 28)

### Convert Output label to multiple values

In [0]:
trainY = tf.keras.utils.to_categorical(trainY, num_classes=10)
testY = tf.keras.utils.to_categorical(testY, num_classes=10)

## Build the Graph

In [0]:
#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Reshape data from 2D (28,28) to 3D (28, 28, 1)
model.add(tf.keras.layers.Reshape((28,28,1),input_shape=(28,28,)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

### Apply Convolutional Layers, MaxPooling

In [0]:
#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))

#Add second convolutional layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#Add MaxPooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

### Add layers for Classification

In [0]:
#Flatten the output
model.add(tf.keras.layers.Flatten())

#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add another dropout layer
model.add(tf.keras.layers.Dropout(0.25))

#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [0]:
sgd = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

### Extend TensorBoard Callback to Log Learning Rate

In [0]:
class LRTensorBoard(tf.keras.callbacks.TensorBoard):
    def __init__(self, log_dir):  # add other arguments to __init__ if you need
        super().__init__(log_dir=log_dir)

    def on_epoch_end(self, epoch, logs=None):
        
        #Get Starting Learning Rate
        lr = tf.keras.backend.eval(self.model.optimizer.lr)
        #Get Decay rate
        decay = tf.keras.backend.eval(self.model.optimizer.decay)
        #Compute LR
        lr = lr *(1/(1 + epoch*decay))
        logs.update({'lr': lr})
        super().on_epoch_end(epoch, logs)

### Train the model

In [0]:
tensorboard = LRTensorBoard(log_dir='logs')

In [11]:
#Train the model
history = model.fit(trainX,trainY,          
              validation_data=(testX,testY),
              callbacks=[tensorboard],
              epochs=1,
              batch_size=32)

1875/1875 [==============================] - 158s 84ms/step - loss: 0.1549 - accuracy: 0.9513 - val_loss: 0.0404 - val_accuracy: 0.9871 - lr: 0.0100
